# Анализ данных и процессов
## Лабораторная работа №4. Поток событий без отказов (с очередью).
### Исходные данные.
На сервере есть n каналов передачи сообщений. Среднее время обработки сообщений τ минут. На сервер поступают сообщения в среднем количестве λ сообщений в минуту.  Параметры варианта определяются по формулам: n=3+[(i+j)/8], λ=1+i/4, τ=5/(5+j). Здесь квадратные скобки означают взятие целой части, а i,j - последние цифры зачетки. Для хранения сообщений в очереди на сервере выделено место для $(n+1)$ сообщения. 

In [1]:
i, j, = 2, 3

N = 3 + round((i + j) / 8)
LAMBDA = 1 + i / 4
TAU = 5 / (5 + j)
M = N + 1

print("Количество каналов N:", N)
print("Максимальное количество сообщений в очереди M:", M)
print("Интенсивность потока событий λ:", LAMBDA)
print("Среднее время обслуживания τ:", TAU)

Количество каналов N: 4
Максимальное количество сообщений в очереди M: 5
Интенсивность потока событий λ: 1.5
Среднее время обслуживания τ: 0.625


### Задание 1. Определить основные характеристики сервера.
$P$ - вероятность очереди;

$k$ - среднее число занятых каналов;

$L_{оч}$ - средняя длина очереди;

$L_{сист}$ - среднее число сообщений на сервере;

In [2]:
MU = 1 / TAU
print("Скорость обработки сообщений μ:", MU)
RHO = LAMBDA / MU
print("Интенсивность потока ρ:", RHO)

Скорость обработки сообщений μ: 1.6
Интенсивность потока ρ: 0.9375


In [3]:
import math

p = [math.pow(
    sum(RHO ** i / math.factorial(i) for i in range(N + 1)) +
    (RHO ** (N + 1) * (1 - (RHO / N) ** M)) / (N * math.factorial(N) * (1 - RHO / N)), -1
)]

for i in range(1, N + 1):
    p.append((RHO / i) * p[i - 1])
    
for i in range(N + 1, N + M + 1):
    p.append((RHO / N) * p[i - 1])

print(p)

[0.3911881307216403, 0.3667388725515378, 0.17190884650853336, 0.05372151453391667, 0.01259097996888672, 0.002951010930207825, 0.000691643186767459, 0.00016210387189862318, 3.799309497623981e-05, 8.904631635056206e-06]


In [4]:
import numpy as np

p_reject = p[-1]
q = 1 - p_reject
a = LAMBDA * q
k = RHO * q

l_queue = p[N + 1] * (1 - (M + 1 - M * (RHO / N)) * (RHO / N) ** M) / (1 - RHO / N) ** 2
l_system = k + l_queue

print("Вероятность отказа в обработке p:", p_reject)
print(f"Относительная пропускная способность Q: {q} ({q * 100}%)")
print("Абсолютная пропускная способность A:", a)
print("Среднее количество занятых каналов k:", k)
print("Средняя длина очереди L_оч:", l_queue)
print("среднее число сообщений на сервере L_сист:", l_system)

Вероятность отказа в обработке p: 8.904631635056206e-06
Относительная пропускная способность Q: 0.999991095368365 (99.9991095368365%)
Абсолютная пропускная способность A: 1.4999866430525475
Среднее количество занятых каналов k: 0.9374916519078421
Средняя длина очереди L_оч: 0.005017104457518852
среднее число сообщений на сервере L_сист: 0.942508756365361


### Задание 2. Написать программу, которая имитирует поведение сервера и вычисляет его основные характеристики.

#### Класс симулятора

In [5]:
from dataclasses import dataclass
from typing import Generator

@dataclass
class State:
    busy_channels: int
    time: float
    queue_count: int


class ServerSimulator:
    def __init__(self, time_limit: float, max_channels: int, queue_length: int):
        self._time_limit = time_limit
        self._max_channels = max_channels
        self._queue_length = queue_length

    def iter_states(self) -> Generator:
        yield (current_state := State(0, 0, 0))

        while current_state.time < self._time_limit:
            next_arrive_delta_time = -np.log(1 - np.random.random()) / LAMBDA
            next_processed_delta_time = np.inf

            if current_state.busy_channels > 0:
                next_processed_delta_time = -np.log(1 - np.random.random()) / (current_state.busy_channels * MU)

            if next_arrive_delta_time < next_processed_delta_time:
                if current_state.busy_channels < self._max_channels:
                    yield (current_state := State(current_state.busy_channels + 1, current_state.time + next_arrive_delta_time, current_state.queue_count))
            
                elif current_state.queue_count < self._queue_length:
                    yield (current_state := State(current_state.busy_channels, current_state.time + next_arrive_delta_time, current_state.queue_count + 1))
                    
                else:
                    yield (current_state := State(current_state.busy_channels, current_state.time + next_arrive_delta_time, current_state.queue_count))
            
            else:
                if current_state.queue_count > 0:
                    current_state.queue_count -= 1
                else:
                    current_state.busy_channels -= 1
                
                yield (current_state := State(current_state.busy_channels, current_state.time + next_processed_delta_time, current_state.queue_count))
                

#### Имитация обработки событий сервером

In [6]:
import itertools

ITERS_COUNT = 10000
TIME_LIMIT = 200

simulator = ServerSimulator(TIME_LIMIT, N, M)

channels_durations = np.zeros((ITERS_COUNT, N + 1))
queue_durations = np.zeros((ITERS_COUNT, M + 1))
all_durations = np.zeros((ITERS_COUNT, N + M + 1))

for index in range(ITERS_COUNT):
    for previous, current in itertools.pairwise(simulator.iter_states()):
        time_delta = current.time - previous.time
        
        channels_durations[index][current.busy_channels] += time_delta
        queue_durations[index][current.queue_count] += time_delta
        all_durations[index][current.busy_channels + current.queue_count] += time_delta
        

#### Вычисление фактических характеристик сервера

In [7]:
prob_busy_channels = np.mean(np.array(channels_durations) / TIME_LIMIT, axis=0)
prob_queue_length = np.mean(np.array(queue_durations) / TIME_LIMIT, axis=0)

p_fact = np.mean(np.array(all_durations) / TIME_LIMIT, axis=0)
k_fact = np.sum(np.array(range(N + 1)) * prob_busy_channels)
l_queue_fact = np.sum(np.array(range(M + 1)) * prob_queue_length)
l_system_fact = k_fact + l_queue_fact

### Задание 3. Сравнить результаты.

In [8]:
import pandas as pd

pd.DataFrame(np.vstack((p, p_fact)).transpose(), columns=["Ожидаемые", "Фактические"])

,Ожидаемые,Фактические
0,0.391188,0.189529
1,0.366739,0.510301
2,0.171909,0.218246
3,0.053722,0.065009
4,0.012591,0.015155
5,0.002951,0.002936
6,0.000692,0.000718
7,0.000162,0.000171
8,0.000038,0.000039
9,0.000009,0.000008


In [9]:
pd.DataFrame(np.array([
    [k, k_fact],
    [l_queue, l_queue_fact],
    [l_system, l_system_fact],
]), index=["k", "L_оч", "L_сист"], columns=["Ожидаемые", "Фактические"])

,Ожидаемые,Фактические
k,0.937492,1.217930
L_оч,0.005017,0.005083
L_сист,0.942509,1.223012


### Вывод
В ходе выполнения лабораторной работы была спроектирована модель системы массового обслуживания с ограниенной очередью. Были теоретически рассчитаны ее ожидаемые характеристики. Затем была написана программа, имитирующая поведение системы, и получены фактические характеристики. Проведен сравнительный анализ ожидаемых и фактических характеристик.